## RNN for Time Series Forecasting
https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras

Recurrent Neural Networks, or RNNs for short, are designed to operate upon sequences of data.

They have proven to be very effective for natural language processing problems where sequences of text are provided as input to the model.

RNNs have also seen some modest success in time series forecasting and speech recognition.

The most popular type of RNN is the Long Short-Term Memory network or LSTM for short.

LSTMs can be used in a model to accept a sequence of input data and make a prediction, such as assign a class label or predict a numerical value like the next value or values in the sequence.

You will use the car sales dataset to demonstrate an LSTM RNN for univariate time series forecasting.

This problem involves predicting the number of car sales per month.

Let’s frame the problem to take a window of the last five months of data to predict the current month’s data.

To achieve this, you define a new function named split_sequence() that will split the input sequence into windows of data appropriate for fitting a supervised learning model, like an LSTM.

LSTMs expect each sample in the dataset to have two dimensions; the first is the number of time steps (in this case, it is 5), and the second is the number of observations per time step (in this case, it is 1).

Because it is a regression-type problem, we will use a linear activation function (no activation function) in the output layer and optimize the mean squared error loss function.

We will also evaluate the model using the mean absolute error (MAE) metric.

In [71]:
import numpy as np
import pandas as pd

In [73]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras import Sequential

In [74]:
import warnings
warnings.filterwarnings('ignore')

In [75]:
tf.get_logger().setLevel('ERROR')

## Load the Dataset

In [76]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return np.asarray(X), np.asarray(y)

In [77]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = pd.read_csv(url, header=0, index_col=0, squeeze=True)

In [78]:
df.head(10)

Month
1960-01     6550
1960-02     8728
1960-03    12026
1960-04    14395
1960-05    14587
1960-06    13791
1960-07     9498
1960-08     8251
1960-09     7049
1960-10     9545
Name: Sales, dtype: int64

In [79]:
# Retrieve the values
values = df.values.astype('float32')
# Specify the window size
n_steps = 5
# Split into samples
X, y = split_sequence(values, n_steps)

In [80]:
# Reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

In [81]:
X

array([[[ 6550.],
        [ 8728.],
        [12026.],
        [14395.],
        [14587.]],

       [[ 8728.],
        [12026.],
        [14395.],
        [14587.],
        [13791.]],

       [[12026.],
        [14395.],
        [14587.],
        [13791.],
        [ 9498.]],

       [[14395.],
        [14587.],
        [13791.],
        [ 9498.],
        [ 8251.]],

       [[14587.],
        [13791.],
        [ 9498.],
        [ 8251.],
        [ 7049.]],

       [[13791.],
        [ 9498.],
        [ 8251.],
        [ 7049.],
        [ 9545.]],

       [[ 9498.],
        [ 8251.],
        [ 7049.],
        [ 9545.],
        [ 9364.]],

       [[ 8251.],
        [ 7049.],
        [ 9545.],
        [ 9364.],
        [ 8456.]],

       [[ 7049.],
        [ 9545.],
        [ 9364.],
        [ 8456.],
        [ 7237.]],

       [[ 9545.],
        [ 9364.],
        [ 8456.],
        [ 7237.],
        [ 9374.]],

       [[ 9364.],
        [ 8456.],
        [ 7237.],
        [ 9374.],
        

In [82]:
# split into train/test
n_test = 12
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(91, 5, 1) (12, 5, 1) (91,) (12,)


## Step 1: Define the Model

In [83]:
# Define model
model = Sequential()
model.add(tfl.LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,1)))
model.add(tfl.Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(tfl.Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(tfl.Dense(1))

## Step 2: Compile the Model

In [84]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

## Step 3: Fit the Model

In [85]:
# Fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))

Epoch 1/350
3/3 - 3s - loss: 347613824.0000 - mae: 14534.6211 - val_loss: 273626784.0000 - val_mae: 12333.8975 - 3s/epoch - 961ms/step
Epoch 2/350
3/3 - 0s - loss: 206761104.0000 - mae: 10680.3291 - val_loss: 167955792.0000 - val_mae: 10634.9658 - 38ms/epoch - 13ms/step
Epoch 3/350
3/3 - 0s - loss: 98178152.0000 - mae: 7993.7803 - val_loss: 117340248.0000 - val_mae: 9360.7275 - 38ms/epoch - 13ms/step
Epoch 4/350
3/3 - 0s - loss: 61082988.0000 - mae: 6131.4238 - val_loss: 76033960.0000 - val_mae: 7070.2739 - 40ms/epoch - 13ms/step
Epoch 5/350
3/3 - 0s - loss: 65856316.0000 - mae: 6403.0586 - val_loss: 66113532.0000 - val_mae: 5745.4526 - 38ms/epoch - 13ms/step
Epoch 6/350
3/3 - 0s - loss: 35067416.0000 - mae: 4516.0068 - val_loss: 27889174.0000 - val_mae: 4339.9458 - 41ms/epoch - 14ms/step
Epoch 7/350
3/3 - 0s - loss: 39777512.0000 - mae: 4661.7812 - val_loss: 61759172.0000 - val_mae: 5656.0630 - 40ms/epoch - 13ms/step
Epoch 8/350
3/3 - 0s - loss: 50142128.0000 - mae: 5430.6719 - val_lo

## Step 4: Evaluate the Model

In [86]:
# Evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, np.sqrt(mse), mae))

MSE: 10679853.000, RMSE: 3268.004, MAE: 2641.625


## Step 5: Make Prediction

In [87]:
# Make a prediction
row = np.asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps))
y_pred = model.predict(row)
print('Predicted: %.3f' % (y_pred))

1/1 [==============================] - 0s 236ms/step
Predicted: 15099.448


In [70]:
y_pred

array([[15787.441]], dtype=float32)